In [160]:
# ensure that the imported .py file will get auto imported and updated whenever there is a change
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [128]:
!pip install PyMuPDF

In [129]:

import fitz
import os

In [161]:

project_path = os.getcwd()
project_data_path = project_path + "/data"
project_processed_data_path = project_path + "/processed"
if not os.path.exists(project_data_path):
    os.makedirs(project_data_path)
if not os.path.exists(project_processed_data_path):
    os.makedirs(project_processed_data_path)

In [162]:
import nltk
from nltk.tokenize import word_tokenize #Used to extract words from documents
from nltk.stem import WordNetLemmatizer #Used to lemmatize words
from nltk.corpus import stopwords
import re
import pandas as pd
pd.options.display.max_colwidth = 1000
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
nltk.download('punkt')
nltk.download('wordnet')

import processing_pdf
pd.options.display.max_colwidth = 2000

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Claud\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Claud\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Claud\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [238]:
# pdf_file =  "An Empirical Survey on Long Document Summarization.pdf"
# pdf_file = "1901.00009v1.pdf"
# pdf_file = "1901.00936v3.pdf"
pdf_file = "an image is worth 16 by 16 words.pdf"
doc, total_text, total_pages = processing_pdf.open_file(project_data_path + "/" + pdf_file)
table_of_content = doc.get_toc()
print("Auto generated table of content:")
display(table_of_content)

# some papers have not table of content
if len(table_of_content) == 0:
    print("The paper has not table of content. Need to use regular expression to map table of content.")
    res = processing_pdf.auto_find_toc(doc)
    print("Not satified the generated result or want to adjust? Build your own table of content by using this template:")
    display(res)



Auto generated table of content:


[[1, '1 Introduction', 1],
 [1, '2 Related Work', 2],
 [1, '3 Method', 3],
 [2, '3.1 Vision Transformer (ViT)', 3],
 [2, '3.2 Fine-tuning and Higher Resolution', 4],
 [1, '4 Experiments', 4],
 [2, '4.1 Setup', 4],
 [2, '4.2 Comparison to State of the Art', 5],
 [2, '4.3 Pre-training Data Requirements', 6],
 [2, '4.4 Scaling Study', 8],
 [2, '4.5 Inspecting Vision Transformer', 8],
 [2, '4.6 Self-supervision', 8],
 [1, '5 Conclusion', 9],
 [1, 'A Multihead Self-attention', 13],
 [1, 'B Experiment details', 13],
 [2, 'B.1 Training', 13],
 [3, 'B.1.1 Fine-tuning', 13],
 [3, 'B.1.2 Self-supervision', 14],
 [1, 'C Additional Results', 14],
 [1, 'D Additional Analyses', 15],
 [2, 'D.1 SGD vs. Adam for ResNets', 15],
 [2, 'D.2 Transformer shape', 16],
 [2, 'D.3 Head Type and class token', 16],
 [2, 'D.4 Positional Embedding', 17],
 [2, 'D.5 Empirical Computational Costs', 18],
 [2, 'D.6 Axial Attention', 19],
 [2, 'D.7 Attention Distance', 20],
 [2, 'D.8 Attention Maps', 20],
 [2, 'D.9 Object

In [239]:
#uncomment this list to customize table-of-content
# table_of_content = [[1, 'I. INTRODUCTION', 1],
#  [1, 'II. SFC BASED ON IPV6 SEGMENT ROUTING', 2],
#  [1, 'III. DESIGN OF THE SRV6 PROXY', 4],
#  [2, 'A. General Concepts and State-of-the-art', 4],
#  [2, 'B. SRNKv1', 5],
#  [2, 'C. SRNKv2', 7],
#  [2, 'D. Implementation of other SR proxy types', 8],
#  [1, 'IV. TESTING ENVIRONMENT', 8],
#  [1, 'V. PERFORMANCE ANALYSIS', 9],
#  [1, 'VII. CONCLUSIONS', 11]]

# separate content into sections
processing_pdf.clear_processed_folder(project_processed_data_path)
title, authors, other_info, abstract = processing_pdf.find_meta_data(doc, table_of_content)
ds, json_dict = processing_pdf.separate_content(total_text, table_of_content)
processing_pdf.save_dataframe(ds, json_dict, project_processed_data_path,  pdf_file.rsplit(".", 1)[0])
# extract images
processing_pdf.find_images(doc, table_of_content, total_pages, project_processed_data_path)

# display(ds)
print(f"\nTitle: {title}")
print(f"\nAuthors:{authors}")
print(f"\nOther info: {other_info}")
print(f"\nAbstract:{abstract}")


starting looking for all the sections according to the provided section title info...
save the dataframe to d:\Start_Dec_19_2023\application\NUS Course\Semester_1\Neural Network and Deep Learning\PDF_Conversion/processed/an image is worth 16 by 16 words.csv
save the dataframe to d:\Start_Dec_19_2023\application\NUS Course\Semester_1\Neural Network and Deep Learning\PDF_Conversion/processed/an image is worth 16 by 16 words.json

Title: AN IMAGE IS WORTH 16X16 WORDS:TRANSFORMERS FOR IMAGE RECOGNITION AT SCALE

Authors:Alexey Dosovitskiy∗,†, Lucas Beyer∗, Alexander Kolesnikov∗, Dirk Weissenborn∗,Xiaohua Zhai∗, Thomas Unterthiner, Mostafa Dehghani, Matthias Minderer,Georg Heigold, Sylvain Gelly, Jakob Uszkoreit, Neil Houlsby∗,†

Other info: ∗equal technical contribution, †equal advisingGoogle Research, Brain Team{adosovitskiy, neilhoulsby}@google.com

Abstract: While the Transformer architecture has become the de-facto standard for naturallanguage processing tasks, its applications to comp

Title: AN IMAGE IS WORTH 16X16 WORDS:TRANSFORMERS FOR IMAGE RECOGNITION AT SCALE

Authors:Alexey Dosovitskiy∗,†, Lucas Beyer∗, Alexander Kolesnikov∗, Dirk Weissenborn∗,Xiaohua Zhai∗, Thomas Unterthiner, Mostafa Dehghani, Matthias Minderer,Georg Heigold, Sylvain Gelly, Jakob Uszkoreit, Neil Houlsby∗,†

Other info: ∗equal technical contribution, †equal advisingGoogle Research, Brain Team{adosovitskiy, neilhoulsby}@google.com

Abstract: While the Transformer architecture has become the de-facto standard for naturallanguage processing tasks, its applications to computer vision remain limited. Invision, attention is either applied in conjunction with convolutional networks, orused to replace certain components of convolutional networks while keeping theiroverall structure in place. We show that this reliance on CNNs is not necessaryand a pure transformer applied directly to sequences of image patches can performvery well on image classiﬁcation tasks. When pre-trained on large amounts ofdata

In [ ]:
import json

# open json file
with open(project_processed_data_path + "/an image is worth 16 by 16 words.json") as f:
    data = json.load(f)
    for item in data:
        display(item)